# This is Jessica's artist similarity graph, edited. 

Basically it does what it originally was intended to do, except after it makes the graph of similar artists, it creates a sparse similarity matrix and runs a (likewise sparse) cosine similarity on it. It then renames the indices for both with the appropriate ID names. 

From there it does a left join of the sparse cosine similarity matrix onto the artist names and ids, resets the columns to be numeric indices (this is so it'll work more readily with my cosine-similarity-weighting script), and saves the output to a .csv file. 

In [181]:
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval
from sys import getsizeof
import pickle

In [2]:
df = pd.read_csv("../EDA/millionsong_subset.csv")

In [7]:
artists = df[['artist_id', 'artist_name', 'similar_artists']]
artists['artist_name'] = artists['artist_name'].str[2:]
artists['artist_name'] = artists['artist_name'].str[:-1]
artists.head()

C:\Users\samue\AppData\Local\Temp\ipykernel_3632\3647394778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artists['artist_name'] = artists['artist_name'].str[2:]
C:\Users\samue\AppData\Local\Temp\ipykernel_3632\3647394778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artists['artist_name'] = artists['artist_name'].str[:-1]


,artist_id,artist_name,similar_artists
0,b'ARD7TVE1187B99BFB1',Casual,"[b'ARV4KO21187FB38008', b'ARWHM281187FB3D381',..."
1,b'ARMJAGH1187FB546F3',The Box Tops,"[b'ARSZWK21187B9B26D7', b'ARLDW2Y1187B9B544F',..."
2,b'ARKRRTF1187B9984DA',Sonora Santanera,"[b'ARFSJUG11C8A421AAD', b'AR8SD041187FB36015',..."
3,b'AR7G5I41187FB4CE6C',Adam Ant,"[b'AR4R0741187FB39AF2', b'AR0D7K21187B9AD14E',..."
4,b'ARXR32B1187FB57099',Gob,"[b'ARUA62A1187B99D9B0', b'ARHJFFY1187B98BA76',..."


In [8]:
artists_graph = nx.Graph()

In [185]:
for _, row in artists.iterrows():
    #print([row["artist_id"], row["artist_name"]])
    
   artists_graph.add_node(row["artist_id"], name=row["artist_name"])

["b'ARWVOHH1187FB5974B'", "Screamin' Jay Hawkins"]
["b'AR4MCB01187B98BF5C'", 'Atman']
["b'ARIQZ2V1187B98E546'", 'A-Studio']
["b'ARFAKQ01187B990163'", 'Georges Delerue']
["b'ARAEXD61187B9B7543'", 'Bellatrax feat. Tina Cousins']
["b'ARP4QQS1187B998E46'", 'Forensics']
["b'AROSI8L1187B9ACD43'", 'Ice Cube']
["b'ARH1UTX1187FB50CAA'", 'Bobby Capo']
["b'ARFLHWV1187FB4E77F'", 'Sandy Posey']
["b'AROSI8L1187B9ACD43'", 'Ice Cube']
["b'ARPMJGA11F4C83BDFE'", 'US Coast Guard Band']
["b'ARFCOGY1187FB454B4'", 'The Winans']
["b'ARY1QJF1187B98EF3F'", 'Kathleen Madigan']
["b'ARN3O411187FB4D859'", 'Stephan Micus']
["b'ARRRVMG1187FB532AC'", 'Radney Foster']
["b'AR9YS621187FB5AC07'", 'The Clark Sisters']
["b'ARL1QL91187B994B08'", 'A Challenge Of Honour']
["b'ARXB7B21187B98BEF0'", 'Marisela']
["b'ARPOQKS1187B988FE2'", 'Ed Calle']
["b'ARTPVDV1187B9A61F1'", 'Johnny Keating & All Stars']
["b'AROG07L1187FB4C826'", 'Boards of Canada']
["b'AR06IFV1187B9977EC'", 'MU330']
["b'ARIWB161187B9AA1D5'", 'Simon & Garfunkel'

["b'AR3EABF1187B98AD35'", 'Sophie Tucker']
["b'ARI4NXX1187FB59DF2'", 'Raze']
["b'ARTP2PK1187B98FECA'", 'Quinteto Violado']
["b'AR12MUW1187B9A5DF4'", 'Ricardo Lemvo']
["b'ARMHR011187B9A62B5'", 'Joe Bataan']
["b'AR02IU11187FB513F2'", 'Tito Puente']
["b'AR4YY6D1187B9AEC44'", 'Spooky Tooth']
["b'ARABDBR11E2835EADC'", 'Larytta']
["b'ARGAWJY11F50C50904'", 'Jane Winther']
["b'AR7DFJP1187B992F13'", 'Juana Molina']
["b'AR5EQRI1187FB41287'", 'Kuba']
["b'ARZTSMH122988F522A'", 'Reginaldo Bessa']
["b'ARYQQSV1252CCF982A'", 'John Hammond']
["b'ARSKZAQ1269FCD50EF'", 'Brice Kapel']
["b'ARNAIMI1187FB5733C'", 'Rick Ross']
["b'AR02YGA1187B9B8AC4'", 'Bersuit Vergarabat']
["b'ARJQ8WU1187FB37FCD'", 'Patrick St-Eloi']
["b'ARMPRI31187FB4F7D9'", 'Vangelis']
["b'ARELA4P1187B991507'", 'Charles Williams']
["b'ARDWS2V1187B99AED0'", 'Mazz']
["b'ARAI2ZO1187B9A0536'", 'Catupecu Machu']
["b'AREP0H61187FB443CF'", 'Grayson Capps']
["b'ARUQZUY1241B9CEB3E'", 'Asure']
["b'ARWD82K1187B98CFF2'", 'Cracker']
["b'AREBCAO1187B994

["b'ARCHMYZ11EB9C820A9'", 'Ada Jane']
["b'ARNARI81187FB3C9A7'", 'Flora Purim']
["b'ARPYQNG1187FB4AF63'", 'Virgil Moorefield']
["b'ARZAHDN1187B9B4BED'", 'Ice_ Lucky Luciano_ Filero']
["b'ARA7WV51187FB532B0'", 'GG Allin']
["b'AR0Q5531187FB45143'", 'Dallas Holm']
["b'ARYGXKQ11E2835CE3B'", 'Styles P']
["b'ART62WC1187FB47E80'", 'The Game feat. Jim Jones Camron & Bezell']
["b'ARML8X41187B990644'", 'Simple Minds']
["b'AR2KSTE1187FB57A36'", 'Petra']
["b'ARCEEC91187B9AC4FB'", "Alexander O'Neal"]
["b'ARMUCXW11F50C4BFEC'", 'Bionikworld']
["b'ARQ4OZM1187B991229'", 'Jean Segurel']
["b'ARKQ6QN1187B9B016D'", 'Photek']
["b'AROLZEM1187B9A805E'", 'Ashes to Ashes']
["b'ARRDRI71187FB46EAA'", 'Band Of Horses']
["b'AR21JV01187B999138'", 'Bill Murk']
["b'ARAVPU21187B993957'", 'Don & Juan']
["b'ARML8X41187B990644'", 'Simple Minds']
["b'ART9Z1P1187FB36C94'", 'Edmundo Ros']
["b'ARNDFTA125FEF32B22'", 'Noemi']
["b'ARTWWNA1187B9B3BD7'", 'Frederik']
["b'AR6EVQA1187B9AA1DF'", 'Cocoa Tea']
["b'ARSF0K11187B9AF319'", '

["b'ARV8T9T1187B99F3F4'", 'Amorphis']
["b'ARHIGWY11F4C8425D5'", "Mr.De'"]
["b'ARWA6NT1187FB3F15C'", 'The Cure']
["b'ARYTJPJ1187FB4ABCC'", 'Cliff Edwards/Disney Studio Chorus']
["b'ARM5JLS1187FB3615B'", 'Tammy Wynette']
["b'AR040QX1187FB4CFE1'", 'Alexisonfire']
["b'ARQXM2T1187B98FF32'", 'Sonny James']
["b'AR7NNKG1187FB4AA1C'", 'Charly Garc\\xc3\\xada']
["b'ARUTP601187FB388AC'", 'The Elders']
["b'ARRZZRW1187B9A06E3'", 'Rachel Portman']
["b'ARA82JG1187FB36131'", 'Charlie Haden']
["b'ARN76SM1187FB386BA'", 'Walter Wanderley Trio']
["b'AR3AIEI1187B992D8D'", 'Marilyn Horne']
["b'ARPB4TX1187B98DDEA'", 'Marga Gomez']
["b'ARDV4XV1187FB5A3F4'", 'Apache Indian']
["b'AR7SF1N1187B9A91D1'", 'John Berry']
["b'AR1XZ0N1187B9941D5'", 'Noah Lewis']
["b'ARMPRI31187FB4F7D9'", 'Vangelis']
["b'ARXLX5L1187FB47BED'", 'Crazy Horse']
["b'ARCVMYS12454A51E6E'", "The Sun Harbor's Chorus-Documentary Recordings"]
["b'AR189B91187B9B8D9C'", 'Gustavo Cerati']
["b'ARNHMFD1187FB3B3F6'", 'LL Cool J']
["b'ARPK1CM1187B9B6B9C'

["b'AR3UJPY1187FB5017B'", 'Patience Dabany']
["b'AREMPZG1187B99D246'", 'Marcio Faraco']
["b'ARJQ8WU1187FB37FCD'", 'Patrick Saint-Eloi']
["b'ARPWJAM11F4C846DF4'", 'Akissa']
["b'AR223M11187B9ADDD6'", 'Marty Robbins']
["b'ARWC1WN1187FB516CE'", 'Eric B. & Rakim']
["b'AR1RHCO1187B9AF0BF'", 'Vince Guaraldi']
["b'AR3OYPP1187B9952E3'", 'Humanzi']
["b'AR12F2S1187FB56EEF'", 'Aerosmith']
["b'ARP39OU1187FB4D543'", 'Aswad']
["b'ARYJ6RP1187B9B3827'", 'Cindy Morgan']
["b'ARTHEAP122BCFCD0B2'", 'DJ X-Change']
["b'AR2SMD91187B9B0582'", 'Mighty Mighty Bosstones']
["b'AR7WZ1Y1187B99D10B'", 'Isabelle Adjani']
["b'ARSYHZ21187B99F5F5'", 'Tina Ann']
["b'AR3DYAD1187FB3964E'", 'Chet Atkins']
["b'ARXAFO11187B9AEBFE'", 'Urge Overkill']
["b'AR109IQ1187B98E487'", 'JK']
["b'ARMB95F1187B9B5B93'", "Cam'Ron"]
["b'ARDVZTE1187FB5A0A1'", 'Harry Connick_ Jr.']
["b'AROLMXS1241B9C6915'", 'An Cafe']
["b'ARM4WGN1187FB3C746'", 'Kyle Eastwood']
["b'ARDK6QW1187B9B8D85'", 'U.K. Subs']
["b'ARGNZOL1187FB5CE7C'", 'Fernando Ortega']
[

["b'ARB78VW1187B98F81C'", 'George Nooks']
["b'AR2VJMU1187B9947AF'", 'Donna Fargo']
["b'ARUME5Q1187B9B1AA7'", 'Kristine W.']
["b'AREN1A11187FB527EF'", 'Roger Creager']
["b'AR9C8IE1187B9ACC99'", 'System of a Down']
["b'AROF4LP1187FB41C51'", 'Selena']
["b'AR9W3X91187FB3994C'", 'Phil Collins']
["b'ARPBTRI1187FB52457'", 'The Game']
["b'ARFQ7RR1187FB3CC0A'", 'Badmarsh & Shri']
["b'ARPXDY61187B9AD0A8'", 'Clannad']
["b'ARL3VGT1187FB40E8E'", 'Los Fabulosos Cadillacs']
["b'ARYFXQ21187FB4DAEF'", 'Poncho Sanchez']
["b'ARML3X41187FB35F2E'", 'Jason Mraz']
["b'ARYN8YT1187FB38396'", 'Los Iracundos']
["b'ARDZRMF122988F7633'", 'Session Americana']
["b'ARMG34A1187FB548A8'", 'Thomas Dolby']
["b'ARRT66G1187FB3DDF9'", 'Freddie King']
["b'ARLOIDN1187B9B060D'", 'DJ Guy']
["b'ARV21JS1187B99E516'", 'Jerry Lewis']
["b'ARN03F71187FB4E3F4'", 'Ayo']
["b'AR14CJ91187FB3A994'", 'Eddy Arnold']
["b'ARYDFJA1241B9CBFF2'", 'Jacques Douai']
["b'AR8CT8U1187FB4D803'", "Dexy's Midnight Runners"]
["b'AR3ZV061187FB3C821'", 'Movi

["b'ARI648V1187B9B5379'", 'Michael Stanley Band']
["b'ARMIQRB12298900AFB'", 'Mar Azul']
["b'ARFCOGY1187FB454B4'", 'The Winans']
["b'AR8UA7Z1187B9AD9C9'", '\\xc3\\x89tienne Daho']
["b'ARAILOU1241B9CA8CD'", "Punk'd Royal"]
["b'AR1WDRJ1187B9BA4B8'", 'Buzzov\\xc2\\xac"en']
["b'AR1NJEP1187FB4F4C1'", '22-20s']
["b'ARW74FW1187FB554AD'", 'Cookies']
["b'AR633SY1187B9AC3B9'", 'Weezer']
["b'ARSDFYB11F50C4F736'", 'Cutting Edge']
["b'ARABFU01187FB46712'", 'STEVE CAMP']
["b'AREN9121187B99D57A'", 'Elvira']
["b'AROC6N61187FB40B14'", 'Steven Halpern']
["b'ARNUPIM11F4C83BD4D'", 'Melt Banana']
["b'AR2S4CA1187B99632E'", 'FFH']
["b'ARC3EZO1187B9B9314'", 'Angels']
["b'AR828WL1187FB47E81'", 'M']
["b'ARC3WPR1187B9A183E'", 'Non Stop DJs']
["b'ARA65601187FB57BD9'", 'Roger Sanchez']
["b'ARE8V001187B994A56'", 'Guy Forsyth']
["b'AROI96Z1187B9A5304'", 'Mish Mash']
["b'ARX83ZF1187FB3B66A'", 'Helen Hobson']
["b'AR2PT4M1187FB55B1A'", 'Los Chichos']
["b'AR358YS1187FB44A29'", 'The Jordanaires & Orion']
["b'ARARFT81187FB

In [10]:
for _, row in artists.iterrows():
    sim_id = literal_eval(row["similar_artists"])
    for similar_id in sim_id:
        
        edge1 = (row["artist_id"] , similar_id)
        edge2 = (similar_id, row["artist_id"])

        if edge1 not in artists_graph.edges and edge2 not in artists_graph.edges:
            artists_graph.add_edge(row["artist_id"], similar_id)

In [24]:
test_list = list(artists_graph.nodes)
test_edge_list = list(artists_graph.edges)

In [25]:
test_list[0]

"b'ARD7TVE1187B99BFB1'"

In [26]:
len(test_list)

113339

In [19]:
#sorted_nodes = sorted(artists_graph.nodes())

adjacency_matrix = nx.to_scipy_sparse_array(artists_graph, nodelist= artists_graph.nodes())

TypeError: '<' not supported between instances of 'bytes' and 'str'

In [13]:
type(adjacency_matrix)

scipy.sparse._csr.csr_array

In [14]:
cosine = cosine_similarity(adjacency_matrix, dense_output = False)

In [15]:
type(cosine)

scipy.sparse._csr.csr_array

In [16]:
cosine.shape

(113339, 113339)

In [ ]:
#artist_names = {row["artist_id"]: row["artist_name"] for _, row in artists.iterrows()}

#similarity_df = pd.DataFrame(cosine, index = sorted_nodes, columns= sorted_nodes)

In [17]:
cosine_df = pd.DataFrame.sparse.from_spmatrix(cosine)

In [32]:
cosine_df = cosine_df.set_axis(test_list,axis=1)
cosine_df = cosine_df.set_axis(test_list,axis=0)

,b'ARD7TVE1187B99BFB1',b'ARMJAGH1187FB546F3',b'ARKRRTF1187B9984DA',b'AR7G5I41187FB4CE6C',b'ARXR32B1187FB57099',b'ARKFYS91187B98E58F',b'ARD0S291187B9B7BF5',b'AR10USD1187B99F3F1',b'AR8ZCNI1187B9A069B',b'ARNTLGG11E2835DDB9',...,b'ARNY0LQ11A348F09A8',b'ARUEQZW12086C16257',b'ARVI3L41187B9B1018',b'ARK230U1187FB3A850',b'ARB60FT1187B9A2253',b'ARQTMNG11C8A415C4B',b'ARJBPFF11E2835CAFF',b'ARSKSSO1187B98ECD0',b'ARQCLLL12454A38BCA',b'ARB83441187FB422CF'
b'ARD7TVE1187B99BFB1',1.0,0,0,0,0,0,0.01,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'ARMJAGH1187FB546F3',0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'ARKRRTF1187B9984DA',0,0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'AR7G5I41187FB4CE6C',0,0,0,1.0,0,0,0,0,0.03,0,...,0,0,0,0,0,0,0,0,0,0
b'ARXR32B1187FB57099',0,0,0,0,1.0,0,0,0.02,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'ARQTMNG11C8A415C4B',0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
b'ARJBPFF11E2835CAFF',0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
b'ARSKSSO1187B98ECD0',0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
b'ARQCLLL12454A38BCA',0,0,0,0,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [31]:
getsizeof(cosine_df)/(1024**2)

310.93831634521484

In [44]:
test = cosine_df.loc[b'ARD7TVE1187B99BFB1']

In [55]:
goodlist = test[test != 0]

In [59]:
goodlist.sort_values(ascending=False)

b'ARD7TVE1187B99BFB1'         1.0
b'ARJOD1M1187B9B7B6B'    0.707107
b'AR0TKGL1187B99C1FF'         0.5
b'ARQH3R11187B991DB5'         0.5
b'AR8GJA41187B999863'         0.5
                           ...   
b'ARCMYSA1187FB4A8E8'         0.1
b'ARSQDUN1187B98D7D7'         0.1
b'ARNH6Q11187FB40FDD'    0.098058
b'ARLUNSX11F50C4FD4B'    0.093659
b'ARM2EJV1187B98EA13'    0.089803
Name: b'ARD7TVE1187B99BFB1', Length: 177, dtype: Sparse[float64, 0]

In [61]:
in_subset_artists = artists[['artist_id','artist_name']]

In [128]:
use = in_subset_artists.merge(cosine_df, left_on = "artist_id", right_index=True)
#use = use.set_index('artist_id')

In [72]:
use

,artist_id,artist_name,b'ARD7TVE1187B99BFB1',b'ARMJAGH1187FB546F3',b'ARKRRTF1187B9984DA',b'AR7G5I41187FB4CE6C',b'ARXR32B1187FB57099',b'ARKFYS91187B98E58F',b'ARD0S291187B9B7BF5',b'AR10USD1187B99F3F1',...,b'ARNY0LQ11A348F09A8',b'ARUEQZW12086C16257',b'ARVI3L41187B9B1018',b'ARK230U1187FB3A850',b'ARB60FT1187B9A2253',b'ARQTMNG11C8A415C4B',b'ARJBPFF11E2835CAFF',b'ARSKSSO1187B98ECD0',b'ARQCLLL12454A38BCA',b'ARB83441187FB422CF'
0,b'ARD7TVE1187B99BFB1',Casual,1.0,0,0,0,0,0,0.01,0,...,0,0,0,0,0,0,0,0,0,0
1,b'ARMJAGH1187FB546F3',The Box Tops,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,b'ARKRRTF1187B9984DA',Sonora Santanera,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,b'AR7G5I41187FB4CE6C',Adam Ant,0,0,0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,b'ARXR32B1187FB57099',Gob,0,0,0,0,1.0,0,0,0.02,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,b'AR4C6V01187FB3BAF4',Moonspell,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,b'AR9JLBU1187B9AAEC4',Danny Williams,0,0.01,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,b'ARS1DCR1187B9A4A56',Winston Reedy,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,b'ARAGMIV11F4C843F78',"Myrick ""Freeze"" Guillory",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
good_col_list = ['artist_id','artist_name'] + use['artist_id'].tolist()

In [101]:
use_with_relevant_cols = use[good_col_list]

In [102]:
use_with_relevant_cols

,artist_id,artist_name,b'ARD7TVE1187B99BFB1',b'ARMJAGH1187FB546F3',b'ARKRRTF1187B9984DA',b'AR7G5I41187FB4CE6C',b'ARXR32B1187FB57099',b'ARKFYS91187B98E58F',b'ARD0S291187B9B7BF5',b'AR10USD1187B99F3F1',...,b'ARUUP4L1187B9B72EB',b'ARI4S0E1187B9B06C0',b'AROIHOI122988FEB8E',b'ARQ91R31187FB38A88',b'ARDK0551187FB5AC48',b'AR4C6V01187FB3BAF4',b'AR9JLBU1187B9AAEC4',b'ARS1DCR1187B9A4A56',b'ARAGMIV11F4C843F78',b'ARYXOV81187B99831D'
0,b'ARD7TVE1187B99BFB1',Casual,1.0,0,0,0,0,0,0.01,0,...,0,0,0,0,0,0,0,0,0,0
1,b'ARMJAGH1187FB546F3',The Box Tops,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.01,0,0,0
2,b'ARKRRTF1187B9984DA',Sonora Santanera,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,b'AR7G5I41187FB4CE6C',Adam Ant,0,0,0,1.0,0,0,0,0,...,0.04,0,0,0,0,0,0,0,0,0
4,b'ARXR32B1187FB57099',Gob,0,0,0,0,1.0,0,0,0.02,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,b'AR4C6V01187FB3BAF4',Moonspell,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.0,0,0,0,0
9996,b'AR9JLBU1187B9AAEC4',Danny Williams,0,0.01,0,0,0,0,0,0,...,0,0,0,0.01,0,0,1.0,0,0,0
9997,b'ARS1DCR1187B9A4A56',Winston Reedy,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0
9998,b'ARAGMIV11F4C843F78',"Myrick ""Freeze"" Guillory",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0


In [138]:
use_with_relevant_cols = use_with_relevant_cols.set_axis(range(-2,len(use_with_relevant_cols.columns)-2),axis=1)

In [139]:
use_with_relevant_cols = use_with_relevant_cols.rename(mapper = {-2: 'artist_id', -1: 'artist_name'}, axis=1)

In [182]:
getsizeof(use_with_relevant_cols)/(1024**2)

74.43096542358398

In [184]:
are_you_sure = False

if are_you_sure: 
    with open('relevant_artist_columns', 'wb') as f: 
        pickle.dump(use_with_relevant_cols, f)

In [176]:
def similar_artists_id_and_cossim(df, index): 
    row = df.iloc[index]
    sorted_row = row[2:-1].sort_values(ascending=False)
    outlist = [(sorted_row.index[i], sorted_row.iloc[i], df['artist_name'].iloc[sorted_row.index[i]]) for i in range(len(sorted_row))]
    return outlist

In [177]:
similar_artists_id_and_cossim(use_with_relevant_cols,2275)

[(2275, np.float64(1.0000000000000007), 'The Beatles'),
 (9134, np.float64(1.0000000000000007), 'The Beatles / Tony Sheridan'),
 (4245, np.float64(0.2600000000000002), 'George Harrison'),
 (441, np.float64(0.2600000000000002), 'George Harrison'),
 (9696, np.float64(0.2600000000000002), 'George Harrison'),
 (5448, np.float64(0.22000000000000014), 'Ray Davies'),
 (6305, np.float64(0.22000000000000014), 'Ray Davies'),
 (702, np.float64(0.20000000000000012), 'The Animals_ Sonny Boy Williamson'),
 (7637, np.float64(0.1800000000000001), 'Marshall Crenshaw'),
 (6706, np.float64(0.1800000000000001), 'Marshall Crenshaw'),
 (6777, np.float64(0.1700000000000001), 'Dwight Twilley'),
 (2055, np.float64(0.1700000000000001), 'Dwight Twilley'),
 (8468, np.float64(0.1700000000000001), 'Electric Light Orchestra'),
 (4975, np.float64(0.1700000000000001), 'Electric Light Orchestra'),
 (1903, np.float64(0.1600000000000001), 'Small Facers'),
 (7978, np.float64(0.1600000000000001), 'The Doors'),
 (7025, np.f

In [173]:
type(use_with_relevant_cols)

pandas.core.frame.DataFrame